In [ ]:
import importlib

import numpy as np
import random
import time
import pandas as pd
import os
import random
from tqdm import tqdm


# Signals
from scipy.stats import mannwhitneyu
from scipy import signal
from scipy.fft import fft, fftshift
from scipy.signal import welch
import scipy.signal

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)  


from matplotlib.colors import Normalize, LogNorm

import auxlib; importlib.reload(auxlib)

In [ ]:
def task_labels(header):	
	t0_labels = []
	t1_labels = []
	t2_labels = []
	for task in range(len(header['annotations'])):
		if header['annotations'][task][2] == 'T1':
			t1_labels.append(header['annotations'][task][0])
		elif header['annotations'][task][2] == 'T2':
			t2_labels.append(header['annotations'][task][0])
		else:
			t0_labels.append(header['annotations'][task][0])
	return t0_labels, t1_labels, t2_labels

In [ ]:
def psd_percentage(x, fs, nfft):
    
    N = nfft
    X = abs(fft(x, n=nfft))/N
    Pxx = X**2
    X = X[0:int(N/2)+1]
    X[1:int(N/2)] = 2*X[1:int(N/2)] # duplicate except 0
    
    Pxx = Pxx[0:int(N/2)+1]
    
    Pxx[1:int(N/2)] = 2*Pxx[1:int(N/2)]

    #Psd = (1/fs*N)*abs(fft(x, n=nfft))**2
    #Psd = Psd[0:int(N/2)+1]
    #Psd[1:int(N/2)] = 2*Psd[1:int(N/2)] # duplicate except 0
    
    f, S = scipy.signal.periodogram(x, fs,  nfft=nfft, scaling='spectrum')

    #psd_sum = sum(Psd)
    
    #Psd_per= Psd/psd_sum
    
    #Psd_per= (nfft/fs)*(Psd/psd_sum)
    
    #f = (fs/N)*np.arange(0, int(N/2)+1, 1) = S
    
    Psd_per = S
    
    psd_sum = sum(Psd_per)
    
    Psd_per= Psd_per/psd_sum
    
    return Psd_per, Pxx, X, f

In [ ]:
def notch_filter(data, f0, Q, fs):
    # IIR notch filter using signal.iirnotch
    b, a = signal.iirnotch(f0, Q, fs)

    # Compute magnitude response of the designed filter
    freq, h = signal.freqz(b, a, fs=fs)

    return signal.filtfilt(b, a, data)

In [ ]:
def butter_highpass_filtering(data_channel, order, f_cutoff, fs):
    
    butter_filter = signal.butter(N=order, Wn=f_cutoff, output='sos', fs=fs, btype='highpass')
    
    high_pass_filtered_eeg = signal.sosfiltfilt(butter_filter, data_channel)

    return high_pass_filtered_eeg

In [ ]:
def denoising_notch(session, task, f0, Q, fs):
    
    data, signal_headers, header = auxlib.loadEEG(subject=session, record=task)
    t0_labels, t1_labels, t2_labels = task_labels(header)
    time = np.linspace(0, len(data[0])/160, len(data[0]))
            
    # filtering noise
    for i in range(0, data.shape[0]):
        data[i] = notch_filter(data[i], f0=60, Q=30, fs=160)
                
        # Save Data
        #data_t = np.transpose(data)
        #pd.DataFrame(data_t, ).to_csv(f'media/S{session:03d}R{run:02d}_denoised_{f0}_Hz.csv')
    
    return data, t0_labels, t1_labels, t2_labels

In [ ]:
def task_fragmentation(data, channel, t0_labels, t1_labels, t2_labels, t_left, t_right):
    data_t0 = []
    data_t1 = []
    data_t2 = []
    
    for i in range(len(t0_labels)):
        t0_index = int(t0_labels[i]*160)
        delta = int(4.1*160)
        t0_segment = data[channel, t0_index: t0_index+delta]
        data_t0.append(t0_segment)
    data_t0 = np.array(data_t0)
    
    
    for j in range(len(t1_labels)):
        t1_index = int(t1_labels[j]*160)
        delta_left = int(t_left*160)
        delta_right = int(t_right*160)
        t1_segment = data[channel, t1_index-delta_left: t1_index+delta_right]
        data_t1.append(t1_segment)
    data_t1 = np.array(data_t1)
    
    for k in range(len(t2_labels)):
        t2_index  = int(t2_labels[k]*160)
        delta_left = int(t_left*160)
        delta_right = int(t_right*160)
        t2_segment = data[channel, t2_index-delta_left: t2_index+delta_right]
        data_t2.append(t2_segment)
    data_t2 = np.array(data_t2)
    
    return data_t0, data_t1, data_t2

In [ ]:
def segments_average(data_segments, fs, nfft):
    Np = data_segments.shape[0] 
    Psd_per_average, Pxx, X_aver, f = psd_percentage(data_segments[0], fs, nfft)
    for i in range(Np-1):
        Psd_per, Pxx, X_aver_n, f = psd_percentage(data_segments[i+1], fs, nfft)
        Psd_per_average = Psd_per_average + Psd_per
        X_aver = X_aver + X_aver_n
    return Psd_per_average/Np, X_aver/Np , f


def psd_per_channels(data, t0_labels, t1_labels, t2_labels, fs, nfft):
        Psd_prom_t0_per_channel = []
        Psd_prom_t1_per_channel = []
        Psd_prom_t2_per_channel = []
        
        for i in range(data.shape[0]):
            data_t0, data_t1, data_t2 = task_fragmentation(data=data,channel=i, 
                                                        t0_labels=t0_labels,
                                                        t1_labels=t1_labels, 
                                                        t2_labels=t2_labels, t_left=t_left, t_right=t_right)
            Psd_prom_t0_channel, X_prom_t0_channel, f = segments_average(data_t0, fs=160, nfft=nfft)
            Psd_prom_t1_channel, X_prom_t1_channel, f = segments_average(data_t1, fs=160, nfft=nfft)
            Psd_prom_t2_channel, X_prom_t2_channel, f = segments_average(data_t2, fs=160, nfft=nfft)
            #Adding psd_prom for each channel for each t0, t1, t2
            Psd_prom_t0_per_channel.append(Psd_prom_t0_channel)
            Psd_prom_t1_per_channel.append(Psd_prom_t1_channel)
            Psd_prom_t2_per_channel.append(Psd_prom_t2_channel)
    
        Psd_prom_t0_per_channel = np.array(Psd_prom_t0_per_channel)
        Psd_prom_t1_per_channel = np.array(Psd_prom_t1_per_channel)
        Psd_prom_t2_per_channel = np.array(Psd_prom_t2_per_channel)
    
        return Psd_prom_t0_per_channel, Psd_prom_t1_per_channel, Psd_prom_t2_per_channel, f